<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/translate_overviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np
import time

import re

from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
dataover = pd.read_csv("gdrive/My Drive/TFG/tmdb_spanish_def.csv", sep='#',encoding='utf-8', lineterminator='\n')
taste = pd.read_csv("gdrive/My Drive/TFG/tmdb_spanish_Jaime_def.csv", sep='#', encoding='utf-8')

In [7]:
taste = taste[~taste['id'].str.contains('/')]
taste['id'] = taste['id'].astype(int)

data = taste.merge(dataover[['id','title','overview','genres','crew','cast']], left_on='id', right_on='id')
data = data[~pd.isna(data.overview)]

data = data.dropna(subset=['like'])
data['like'] = data['like'].astype(int)
data.reset_index(inplace=True,drop=True)

In [8]:
!pip install translators

In [9]:
import translators as ts

API = 'google'

def translator_constructor(api):
    if api == 'google':
        return ts.google
    elif api == 'bing':
        return ts.bing
    elif api == 'baidu':
        return ts.baidu
    elif api == 'sogou':
        return ts.sogou
    elif api == 'youdao':
        return ts.youdao
    elif api == 'tencent':
        return ts.tencent
    elif api == 'alibaba':
        return ts.alibaba
    elif api == 'yandex':
        return ts.yandex
    else:
        raise NotImplementedError(f'{api} translator is not realised!')

translator = translator_constructor(API)

def translate(x, lang='en'):
    try:
        return translator(x, 'es', lang)
    except:
        print('KO')
        time.sleep(60)
        return translate(x, lang='en')

In [10]:
data['first_sentence'] = data.overview.progress_apply(lambda x: "".join(re.split('[?.,]', x)[0:2]))
data['first_sentence_en'] = ''

In [ ]:
translator = translator_constructor('google')
data.loc[0:300, 'first_sentence_en'] = data.loc[0:300, 'first_sentence'].progress_apply(lambda x: translate(x))

In [ ]:
data.to_csv("gdrive/My Drive/TFG/overview_translate_1.csv", sep='#', encoding='utf-8')